In [838]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares, minimize
from scipy.misc import logsumexp

## 1. Run GPU DSM

Check the input input file for simulation

In [839]:
f = open('input.dat', 'r')
file_contents = f.read()
print (file_contents)
f.close()

	1
	star
	3
	16 16 16
	5000
	0.0     0.0 0.0 0.0     0.0     0.0     0.0     0.0     0.0
	1
	0
	1
	1.0
	100000



Run the simulation

In [840]:
!./gpu_DSM

device Count 1 
Using device 0: GeForce GTX 780



Architecture is star
simulation parameters:
NK	Beta	N_cha
48	1	5000
deformation tensor:
0	0	0
0	0	0
0	0	0
G(t) calculation is on
simulation time, sync time
100000	1


generating chain conformations on host..done
preparing GPU random number generator parameters..
device random number generator parameters done
preparing GPU chain conformations..
 device constants done

 GPU random generator init: 
  device random generators 1 seeding...done
  device random generators 2 seeding...done
  preparing random number sequence...done
 GPU random generator init done.

 Number of ensemble blocks 1
number of correlator levels	6

  copying chains to device block 1. chains in the ensemble block 5000
 device chains done
init done
G(t) calc...



0	21.7456
1	17.9226
2	16.3738
3	15.4156
4	14.7222
5	14.1776
6	13.7279
7	13.3441
8	13.0092
9	12.7116
10	12.4438
11	12.2008
12	11.9776
13	11.7714
14	11.5798
15	11.4011
16	11.2874
24	10.1949
32	9.42454
40	8.82803
48	8.34176
56	7.93229
64	7.57782
72	7.26546
80	6.9873
88	6.73702
96	6.50922
104	6.30043
112	6.10855
120	5.93039
128	5.8238
192	4.76709
256	4.06329
320	3.54693
384	3.14302
448	2.81652
512	2.5458
576	2.31671
640	2.12013
704	1.95036
768	1.8002
832	1.66693
896	1.54849
960	1.44418
1024	1.39721
1536	0.84539
2048	0.544513
2560	0.367367
3072	0.257904
3584	0.179822
4096	0.130372
4608	0.0935718
5120	0.0734361
5632	0.054022
6144	0.0371781
6656	0.0271026
7168	0.0199337
7680	0.0140756
8192	0.0169546
12288	-0.000467536
16384	0.0043814
20480	0.00135233
24576	0.0015659
28672	-0.00676224
32768	-0.00740397
36864	0.00468943
40960	-0.00252664
45056	-0.00105573
49152	-0.00361568
53248	-0.0028071
57344	0.00133655
61440	0.0123927
65536	-0.000127125


Memory cleanup.. done.
Calculation time: 681599 milliseconds


## 2. Fit $f_d(t)$

We follow only slip-links created by SD from their individual times of creation:

\begin{align}
f_d(t) &= \int\int p^{\textrm{eq}}\left(\tau^{\textrm{CD}}\right) p^{\textrm{cr}}\left(\tau^{\textrm{SD}}\right) e^{-\frac{t}{\tau^{\textrm{CD}}}} e^{-\frac{t}{\tau^{\textrm{SD}}}} \mathrm{d}\tau^{\textrm{CD}} \mathrm{d}\tau^{\textrm{SD}} && \\
&= \int_0^\infty p^{\textrm{eq}}\left(\tau^{\textrm{CD}}\right) e^{-\frac{t}{\tau^{\textrm{CD}}}} \mathrm{d}\tau^{\textrm{CD}}\,\int_0^\infty p^{\textrm{cr}}\left(\tau^{\textrm{SD}}\right) e^{-\frac{t}{\tau^{\textrm{SD}}}} \mathrm{d}\tau^{\textrm{SD}} &&
\end{align}

We give our code $p^{\textrm{eq}}\left(\tau^{\textrm{CD}}\right)$ as an input and would like to know $p^{\textrm{cr}}\left(\tau^{\textrm{SD}}\right)$ that comes out from code by fitting $f_d(t)$.

If we don't know the relaxation spectrum a priori, we can start with the discrete spectrum
\begin{equation}
p^{\textrm{cr}}\left(\tau\right) = \sum_i g_i \delta(\tau - \tau_i)
\end{equation}

\begin{equation}
\int_0^\infty p^{\textrm{cr}}\left(\tau^{\textrm{SD}}\right) e^{-\frac{t}{\tau^{\textrm{SD}}}} \mathrm{d}\tau^{\textrm{SD}} = \sum_i g_i e^{-\frac{t}{\tau_i}}
\end{equation}

We are interested in $p^{\textrm{eq}}\left(\tau\right)$ and we need to convert our discrete fit

\begin{equation}
p^{\textrm{eq}}\left(\tau\right) = \frac{\sum_i g_i \tau_i \delta\left(\tau - \tau_i\right)}{\sum_i g_i \tau_i} = \sum_i g_i' \delta\left(\tau - \tau_i\right)
\end{equation}

**First, read $f_d(t)$ from code**

In [870]:
with open('fdt.dat') as f:
    lines = f.readlines()
    x = np.array([float(line.split()[0]) for line in lines])
    y = np.array([float(line.split()[1]) for line in lines])

tfinal=x[-1]

fig = plt.figure(figsize=(8, 6))

ax1 = fig.add_subplot(111)

ax1.set_title("Entanglement lifetime distribution")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'$f_d(t)$')

ax1.plot(x,y, c='r', label=r'$f_d(t)$')

leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

plt.show()

** Trim data to $t>0.1 \tau_c $ and subsample it $\times 10$ times to speed-up fitting **

In [871]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx
#Remove all zeros from data
mask = y!=0
x=x[mask]
y=y[mask]

#Cut data points on the left where they dont change much
cutoff=find_nearest(x, 1e-2)
x=x[cutoff:]
y=y[cutoff:]
#Subsample data
x=x[0::10]
y=y[0::10]

** Define model function, residuals and mean-squared error **

In [872]:
def fdt(time, params):
    lambdaArr = np.split(params,2)[0]
    gArr = np.split(params,2)[1]/np.sum(np.split(params,2)[1])
    return np.dot(np.exp(-time/lambdaArr), gArr)

def log_fdt(time,params):
    lambdaArr = np.split(params,2)[0]
    gArr = np.split(params,2)[1]/np.sum(np.split(params,2)[1])
    return logsumexp(-time/lambdaArr, b=gArr)

#Vectorize function fdt and log_fdt
fdtvec=np.vectorize(fdt, excluded=['params'])
logfdtvec=np.vectorize(log_fdt, excluded=['params'])

#Define residuals
def residuals_fdt(param):
    return fdtvec(time=x, params=param)-y

def residuals_log_fdt(param):
    #print(logfdtvec(time=x[:-1], params=param)-np.log(y[:-1]))
    if np.any(fdtvec(time=x[:-1], params=param) < 0):
        return np.full(x[:-1].shape,1e8) #Penalty for negative f_d(t)
    else:
        return logfdtvec(time=x[:-1], params=param)-np.log(y[:-1])

def MSE(param):
    return np.dot(residuals_fdt(param),residuals_fdt(param))/np.size(x)

def log_MSE(param):
    return np.dot(residuals_log_fdt(param),residuals_log_fdt(param))/np.size(x)

## Optimizing strategy:
1. Start with some big number of modes, like $n_{modes}=15$
2. Run the least square optimization with standard residuals $y_i-f(x_i)$.
3. Scan through different number of modes down to 1.
4. If fit results have any negative weights, $g_i<0$, ignore
5. If fit results have all positive weights, run the least square optimization with log-residuals
6. Choose the best result among step 5

## Use SciPy numeric least square minimization algorithm

** First, optimize using standard residuals $y_i-f(x_i)$ **

In [873]:
fits_1 = [] #output of fitting function for all tested numbers of modes
successful_fits_1 = [] #number of modes for successful fits
for nmodes in range(1, 15):
    lambdaArrInit=10.0**((np.array(range(nmodes), float) + 1.0)/nmodes*np.log10(tfinal))
    gArrInit=np.full(nmodes, 1.0/nmodes)

    fit = least_squares(residuals_fdt, np.append(lambdaArrInit, gArrInit), xtol=1e-15)
    fits_1.append(fit)
    if fit.success and not np.any(fdtvec(time=x, params=fit.x) < 0):
        successful_fits_1.append(nmodes)

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


In [874]:
successful_fits_1

[1, 2, 3, 4, 5, 8]

** Next, optimize solution which have no negative weights further using log-residuals $\log(y_i)-\log(f(x_i))$ to find best fit for the longest relaxation tail **

In [876]:
fits_2 = [] #output of fitting function for all tested numbers of modes
min_log_SME = log_MSE(fits_1[successful_fits_1[0]-1].x)
best_nmodes = successful_fits_1[0]
for i in successful_fits_1:
    fit = fits_1[i-1]
    print('nmodes\t{0}'.format(i))
    print(fit.message)
    print('Initial guess MSE\t{0}'.format(MSE(np.append(lambdaArrInit, gArrInit))))
    print('Fit MSE\t\t\t{0}'.format(MSE(fit.x)))

    fit2 = least_squares(residuals_log_fdt, fit.x, xtol=1e-14, ftol=1e-14)
    fits_2.append(fit2)

    if fit2.success:
        if log_MSE(fit2.x)<min_log_SME:
            min_log_SME = log_MSE(fit2.x)
            best_fit = fit2
            best_nmodes = i
        print(fit2.message)
        print('First fit log-MSE\t{0}'.format(log_MSE(fit.x)))
        print('Second fit log-MSE\t{0}'.format(log_MSE(fit2.x)))

    print(' ')

nmodes	1
`ftol` termination condition is satisfied.
Initial guess MSE	0.19108647582
Fit MSE			0.000852984467009


`ftol` termination condition is satisfied.
First fit log-MSE	10102785.1797


Second fit log-MSE	13.4228668578
 
nmodes	2
`gtol` termination condition is satisfied.
Initial guess MSE	0.19108647582
Fit MSE			7.17954246683e-05


/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


`xtol` termination condition is satisfied.
First fit log-MSE	72400.7504066


...

Second fit log-MSE	0.0144406585426
 
nmodes	5
`gtol` termination condition is satisfied.
Initial guess MSE	0.19108647582
Fit MSE			5.45457316641e-05


`ftol` termination condition is satisfied.
First fit log-MSE	1.55179535639


Second fit log-MSE	0.00263541610355
 
nmodes	8
`ftol` termination condition is satisfied.
Initial guess MSE	0.19108647582
Fit MSE			5.45376971703e-05


`xtol` termination condition is satisfied.
First fit log-MSE	0.312357585186


Second fit log-MSE	0.000339214522958
 


In [877]:
best_nmodes

8

In [878]:
best_fit.x

array([  1.11159498e+00,   7.11958333e+00,   3.81001099e+01,
         1.85951710e+02,   7.53653571e+02,   2.80125319e+03,
         1.10932766e+04,   4.19757413e+04,   5.63780216e+00,
         8.66442998e-01,   2.42286600e-01,   6.15896212e-02,
         1.26790610e-02,   2.52721050e-03,   3.54905987e-05,
        -6.15852018e-06])

In [880]:
fit = fits_1[8-1]
fit2 = best_fit

fig3 = plt.figure(figsize=(24, 6))

ax0 = fig3.add_subplot(131)

ax0.set_title(r'$p^{cr}\left(\tau\right)$')
ax0.set_xlabel(r'$\lambda$')
ax0.set_ylabel(r'$g$')

ax0.scatter(lambdaArrInit,gArrInit, c='k', label=r'Initial guess')
ax0.scatter(np.split(fit.x,2)[0], np.split(fit.x,2)[1]/np.sum(np.split(fit.x,2)[1]), c='r', label=r'First fit')
ax0.scatter(np.split(fit2.x,2)[0],np.split(fit2.x,2)[1]/np.sum(np.split(fit2.x,2)[1]), c='b', label=r'Second fit')
leg = ax0.legend()
ax0.set_xscale('log')

ax1 = fig3.add_subplot(132)

ax1.set_title("Check results of the fit")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'log residuals')

ax1.plot(x,fdtvec(time=x, params=np.append(lambdaArrInit, gArrInit)), c='k', label=r'Initial guess')
ax1.plot(x,fdtvec(time=x, params=fit.x), c='r', label=r'First fit')
ax1.plot(x,fdtvec(time=x, params=fit2.x), c='b', label=r'Second fit')
ax1.plot(x,y, c='g', label=r'$f_d(t)$')

leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

ax2 = fig3.add_subplot(133)

ax2.set_title(r'$f_d(t)$')
ax2.set_xlabel(r'$t/\tau_c$')
ax2.set_ylabel(r'$f_d(t)$')

ax2.plot(x,fdtvec(time=x, params=np.append(lambdaArrInit, gArrInit)), c='k', label=r'Initial guess')
ax2.plot(x,fdtvec(time=x, params=fit.x), c='r', label=r'First fit')
ax2.plot(x,fdtvec(time=x, params=fit2.x), c='b', label=r'Second fit')
ax2.plot(x,y, c='g', label=r'Simulation data')
leg = ax2.legend()
ax2.set_xscale('log')

plt.show()

### Equilibrium spectrum $p^{\textrm{eq}}\left(\tau\right)$

In [881]:
li=np.split(fit2.x,2)[0]
gi=np.multiply(np.split(fit2.x,2)[0], np.split(fit2.x,2)[1]/np.sum(np.split(fit2.x,2)[1]))/np.dot(np.split(fit2.x,2)[0], np.split(fit2.x,2)[1]/np.sum(np.split(fit2.x,2)[1]))

In [882]:
fig8 = plt.figure(figsize=(8, 6))

ax1 = fig8.add_subplot(111)

ax1.set_title("Multimode $f_d(t)$ fitting")
ax1.set_xlabel(r'$\lambda$')
ax1.set_ylabel(r'$g^\prime$')

ax1.scatter(li,gi, c='r', label=r'Best SciPy fit')
leg = ax1.legend()
ax1.set_xscale('log')

plt.show()

** Shape of the discrete equilibrium spectrum is similar to two linear pieces. We may try to identify them using Ramer-Douglas-Peucker algorithm **

In [883]:
from math import sqrt

In [884]:
def distance(a, b):
    return  sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def point_line_distance(point, start, end):
    if (start == end):
        return distance(point, start)
    else:
        n = abs(
            (end[0] - start[0]) * (start[1] - point[1]) - (start[0] - point[0]) * (end[1] - start[1])
        )
        d = sqrt(
            (end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2
        )
        return n / d

** Find the farthest point from the line connecting first and last point in the spectrum **

In [885]:
dmax = 0.0
index = 0
for i in range(1, len(li) - 1):
    d = point_line_distance((np.log(li[i]),gi[i]), (np.log(li[0]),gi[0]), (np.log(li[-1]),gi[-1]))
    if d > dmax:
        index = i
        dmax = d
(np.log(li[index]),gi[index])

(5.2254870182142259, 0.22956040173293626)

** Define bilinear BSW spectrum **

In [886]:
def biliniear_spectrum(t, params):
    if (t > params[1] and t < params[2]):
        return params[0] + params[4] * np.log(t)
    elif (t > params[2] and t < params[3]):
        return params[0] + params[4] * np.log(params[2]) + params[5] * (np.log(t)-np.log(params[2]))
    else:
        return 0.0

** Initial guess from multimode discrete spectrum **

In [887]:
initial_bsw=(gi[0], li[0], li[index], li[-1],(gi[index]-gi[0])/(np.log(li[index])-np.log(li[0])),(gi[-1]-gi[index])/(np.log(li[-1])-np.log(li[index])))
bilinear_spectrum_vec=np.vectorize(biliniear_spectrum, excluded=['params'])

fig9 = plt.figure(figsize=(16, 6))

ax0 = fig9.add_subplot(111)

lArray=10**(-3+(np.array(range(1001), float)/1000)*10)
ax0.plot(lArray,bilinear_spectrum_vec(t=lArray, params=initial_bsw), c='k')
ax0.scatter(li,gi, c='r', label=r'Best SciPy fit')
ax0.set_xscale('log')
plt.show()